In [1]:
import os
import ccxt
import datetime
from configparser import ConfigParser
from binance.client import Client
from utils.functions_bollinger import *

In [2]:
CREDENTIALS_PATH = os.path.join(os.path.expanduser('~'), ".binance/credentials")
config = ConfigParser()
config.read(CREDENTIALS_PATH)
api_key = config.get('trade', 'API_KEY')
api_secret  = config.get('trade', 'SECRET_KEY')

In [3]:
# Initialize Variables
CANDLE_DURATION_IN_MIN = 15

RSI_PERIOD = 14
RSI_OVERBOUGHT = 55
RSI_OVERSOLD = 45

CCXT_TICKER_NAME = 'DOGE/USDT'
TRADING_TICKER_NAME = 'DOGEUSDT'

INVESTMENT_AMOUNT_DOLLARS = 12
holding_quantity = 0

exchange = ccxt.binance()
binance_client = Client(api_key=api_key, api_secret=api_secret)
DECIMALS_QUANTITY = check_decimals(TRADING_TICKER_NAME, binance_client)

* It is recommend to initialize the bot close to the moment that a candle is closed

In [4]:
# currently_holding = False
# while 1:
#     # STEP 1: FETCH THE CANDLE DATA
#     ticker_data = fetch_data(exchange, CCXT_TICKER_NAME, CANDLE_DURATION_IN_MIN)
    
#     if ticker_data is not None:
#         # STEP 2: COMPUTE THE TECHNICAL INDICATORS & APPLY THE TRADING STRATEGY
#         trade_rec_type = get_trade_recommendation_MACD(ticker_data)
#         print(f'{datetime.now().strftime("%d/%m/%Y %H:%M:%S")} {CCXT_TICKER_NAME} TRADING RECOMMENDATION: {trade_rec_type}')

#         # STEP 3: EXECUTE THE BUY OPERATION
#         if (trade_rec_type == 'BUY' and not currently_holding):
#             print(f'Placing {trade_rec_type} order')
#             trade_successful, holding_quantity = execute_trade(binance_client, trade_rec_type, TRADING_TICKER_NAME, INVESTMENT_AMOUNT_DOLLARS, DECIMALS_QUANTITY, holding_quantity)
#             currently_holding = not currently_holding if trade_successful else currently_holding
#             buy_position = True
            
#             # CHECK SELL OPERATION
#             while buy_position:
#                 ticker_data = fetch_data(exchange, CCXT_TICKER_NAME, CANDLE_DURATION_IN_MIN)
#                 confirm_sell = confirm_sell_operation(ticker_data, weight = 1)
#                 print(confirm_sell)
#                 if (confirm_sell == "SELL"):
#                     print(f'Placing SELL order')
#                     trade_successful, holding_quantity = execute_trade(binance_client, confirm_sell, TRADING_TICKER_NAME, INVESTMENT_AMOUNT_DOLLARS, DECIMALS_QUANTITY, holding_quantity)
#                     currently_holding = not currently_holding if trade_successful else currently_holding
#                     buy_position = not buy_position
                
#                 time.sleep(15)

#         time.sleep(CANDLE_DURATION_IN_MIN*60)
#     else:
#         print(f'Unable to fetch ticker data - {CCXT_TICKER_NAME}. Retrying!!')
#         time.sleep(5)

In [4]:
CCXT_TICKER_NAME = ['LUNA/USDT', 'DOT/USDT', 'DOGE/USDT', 'AVAX/USDT', 'MATIC/USDT']
TRADING_TICKER_NAME = ['LUNAUSDT', 'DOTUSDT', 'DOGEUSDT', 'AVAXUSDT', 'MATICUSDT']

currently_holding = False
while 1:
    for ccxt_ticker_name, trading_ticker_name in zip(CCXT_TICKER_NAME, TRADING_TICKER_NAME):
        # STEP 1: FETCH THE CANDLE DATA
        ticker_data = fetch_data(exchange, ccxt_ticker_name, CANDLE_DURATION_IN_MIN)
        decimals_quantity = check_decimals(trading_ticker_name, binance_client)
        
        if ticker_data is not None:
            # STEP 2: COMPUTE THE TECHNICAL INDICATORS & APPLY THE TRADING STRATEGY
            trade_rec_type = get_trade_recommendation_MACD(ticker_data)
            print(f'{datetime.now().strftime("%d/%m/%Y %H:%M:%S")} {ccxt_ticker_name} TRADING RECOMMENDATION: {trade_rec_type}')

            # STEP 3: EXECUTE THE BUY OPERATION
            if (trade_rec_type == 'BUY' and not currently_holding):
                print(f'Placing {trade_rec_type} order')
                trade_successful, holding_quantity, order_price = execute_trade(binance_client, trade_rec_type, trading_ticker_name, INVESTMENT_AMOUNT_DOLLARS, decimals_quantity, holding_quantity)
                currently_holding = not currently_holding if trade_successful else currently_holding
                buy_position = True
                
                # CHECK SELL OPERATION
                while buy_position:
                    ticker_data = fetch_data(exchange, ccxt_ticker_name, CANDLE_DURATION_IN_MIN)
                    confirm_sell = confirm_sell_operation(ticker_data, 1, exchange, ccxt_ticker_name, order_price)
                    print(confirm_sell)
                    if (confirm_sell == "SELL"):
                        print(f'Placing SELL order')
                        trade_successful, holding_quantity, order_price = execute_trade(binance_client, confirm_sell, trading_ticker_name, INVESTMENT_AMOUNT_DOLLARS, decimals_quantity, holding_quantity)
                        currently_holding = not currently_holding if trade_successful else currently_holding
                        buy_position = not buy_position
                    
                    time.sleep(30)

        else:
            print(f'Unable to fetch ticker data - {ccxt_ticker_name}. Retrying!!')
            time.sleep(5)

    print("Waiting for next candle...")
    time.sleep(CANDLE_DURATION_IN_MIN*60)

01/05/2022 11:00:39 LUNA/USDT TRADING RECOMMENDATION: WAIT
01/05/2022 11:00:40 DOT/USDT TRADING RECOMMENDATION: SELL
01/05/2022 11:00:40 DOGE/USDT TRADING RECOMMENDATION: WAIT
01/05/2022 11:00:42 AVAX/USDT TRADING RECOMMENDATION: WAIT
01/05/2022 11:00:44 MATIC/USDT TRADING RECOMMENDATION: WAIT
Waiting for next candle...
01/05/2022 11:15:45 LUNA/USDT TRADING RECOMMENDATION: WAIT
01/05/2022 11:15:46 DOT/USDT TRADING RECOMMENDATION: WAIT
01/05/2022 11:15:47 DOGE/USDT TRADING RECOMMENDATION: WAIT
01/05/2022 11:15:47 AVAX/USDT TRADING RECOMMENDATION: WAIT
01/05/2022 11:15:49 MATIC/USDT TRADING RECOMMENDATION: WAIT
Waiting for next candle...
01/05/2022 11:30:51 LUNA/USDT TRADING RECOMMENDATION: SELL
01/05/2022 11:30:52 DOT/USDT TRADING RECOMMENDATION: WAIT
01/05/2022 11:30:52 DOGE/USDT TRADING RECOMMENDATION: WAIT
01/05/2022 11:30:53 AVAX/USDT TRADING RECOMMENDATION: WAIT
01/05/2022 11:30:54 MATIC/USDT TRADING RECOMMENDATION: WAIT
Waiting for next candle...
01/05/2022 11:45:56 LUNA/USDT TRAD

In [5]:
holding_quantity

0.21

In [6]:
check_decimals('AVAXUSDT', binance_client)

2

In [7]:
12/58.4

0.20547945205479454

In [4]:
round_up(12/58.4, 2)

0.21

In [9]:
round_up(12/58.4, 4)

0.2055

In [8]:
round(3.129, 2)

3.13

In [5]:
11.1*1.077

11.954699999999999

In [3]:
holding_quantity

NameError: name 'holding_quantity' is not defined